## All the import

In [1]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping


from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle
from keras.layers import Merge

/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which a

## Load the data

In [2]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

In [3]:
def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            

    neg1 = verif

    verif=[]

    verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    # 
    l = len(bdd)
    matrice = np.zeros((l,36,36,1)) #4
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing, true_y

## Calculate the performance

In [4]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

## Create The Model

In [5]:
def get_cnn_network(matrixsize = 6, nbfilter = 24, matrixsize2 = 7, nbfilter2 = 6):    
    k = matrixsize
    # init_weights
    I = np.eye(k)
    M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
    I2 = np.zeros((k,k))
    M2 = np.zeros((k,k))
    for j in range(k):
        I2[:,j] = I[:,k-j-1]
        M2[:,j] = M[:,k-j-1]        
    W = np.zeros((k,k,1,nbfilter))
    W[:,:,0,0] = I
    W[:,:,0,1] = I2
    W[:,:,0,2] = M
    W[:,:,0,3] = M2
    for j in range(4,nbfilter):
        W[:,:,0,j] = np.random.randn(k,k)*0.2
    print('configure cnn network')
 
    model = Sequential()
    model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter)]))    
    model.add(AveragePooling2D(pool_size=(3,3)))
    print(model.output_shape)
    
    k2 = matrixsize2
    I = np.eye(k2)
    M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
    I2=np.zeros((k2,k2))
    M2=np.zeros((k2,k2))
    for j in range(k2):
        I2[:,j] = I[:,k2-j-1]
        M2[:,j] = M[:,k2-j-1]   
    
    Z = np.zeros((k2,k2,nbfilter,nbfilter2))
    
    for u in range(nbfilter):
        Z[:,:,u,0] = I
        Z[:,:,u,1] = I2
        Z[:,:,u,2] = M
        Z[:,:,u,3] = M2
        for p in range(4,nbfilter2):
            Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

    model.add(Conv2D(filters = nbfilter2, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter2)]))
    print(model.output_shape)    
    model.add(Dropout(0.1))
    model.add(Flatten())
        
    return model

In [6]:
def get_cnn_network_seq(nbfilter = 64, kernelsize = 7):        
    model = Sequential()
    model.add(Conv1D(filters = nbfilter, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None)))
    model.add(MaxPooling1D(pool_size=8))
    model.add(Dropout(0.2))
    print(model.output_shape)
    model.add(Flatten())
    return model

In [7]:
def create_model(matrixsize11 = 6, nbfilter11 = 24, matrixsize12 = 7, nbfilter12 = 6, 
                 matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7, nbfilter22 = 6,
                 nbfilter1 = 64, kernel_size1 = 7, nbfilters2 = 64,kernel_size2 = 7,
                 Dense1 = 512, Dense2 = 128, Dense3 = 512, Dense4 = 128, Dense5 = 64):
    training_net=[]
    training_net1 = []
    training_net2 = []
    training_net1.append(get_cnn_network(matrixsize11, nbfilter11, matrixsize12, nbfilter12))
    training_net1.append(get_cnn_network(matrixsize21, nbfilter21, matrixsize22, nbfilter22))
    training_net2.append(get_cnn_network_seq(nbfilter1,kernel_size1))
    training_net2.append(get_cnn_network_seq(nbfilter1,kernel_size1))
    training_net2.append(get_cnn_network_seq(nbfilters2,kernel_size2))
    training_net2.append(get_cnn_network_seq(nbfilters2,kernel_size2))
    
    
    model2 = Sequential()
    model2.add(Merge(training_net2, mode ='concat'))
    model2.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model2.add(BatchNormalization())
    model2.add(Activation('relu'))
    model2.add(Dropout(0.2))
    #model2.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    #model2.add(BatchNormalization())
    #model2.add(Activation('relu'))
    #model2.add(Dropout(0.2))
    
    model1 = Sequential()
    model1.add(Merge(training_net1, mode ='concat'))
    model1.add(Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model1.add(Dropout(0.1))
    model1.add(BatchNormalization())
    model1.add(Activation('relu'))
    
    training_net.append(model1)
    training_net.append(model2)
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
    
    model.add(Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.1))
    if Dense5>0:
        model.add(Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
          
    return model

In [8]:
def create_model2(matrixsize11 = 6, nbfilter11 = 24, matrixsize12 = 7, nbfilter12 = 6, 
                 matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7, nbfilter22 = 6,
                 nbfilter1 = 64, kernel_size1 = 7, nbfilters2 = 64,kernel_size2 = 7,
                 Dense1 = 512, Dense2 = 128, Dense3 = 512, Dense4 = 128, Dense5 = 64):
    training_net=[]
    training_net1 = []
    training_net2 = []
    training_net21 = []
    training_net22 = []
    training_net1.append(get_cnn_network(matrixsize11, nbfilter11, matrixsize12, nbfilter12))
    training_net1.append(get_cnn_network(matrixsize21, nbfilter21, matrixsize22, nbfilter22))
    training_net21.append(get_cnn_network_seq(nbfilter1,kernel_size1))
    training_net21.append(get_cnn_network_seq(nbfilter1,kernel_size1))
    training_net22.append(get_cnn_network_seq(nbfilters2,kernel_size2))
    training_net22.append(get_cnn_network_seq(nbfilters2,kernel_size2))
    
    
    model21 = Sequential()
    model21.add(Merge(training_net21, mode ='concat'))
    model21.add(Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model21.add(BatchNormalization())
    model21.add(Activation('relu'))
    model21.add(Dropout(0.2))
    model22 = Sequential()
    model22.add(Merge(training_net22, mode ='concat'))
    model22.add(Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model22.add(BatchNormalization())
    model22.add(Activation('relu'))
    model22.add(Dropout(0.2))
    training_net2.append(model21)
    training_net2.append(model22)
    model2 = Sequential()
    model2.add(Merge(training_net2, mode ='concat'))
    model2.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model2.add(BatchNormalization())
    model2.add(Activation('relu'))
    model2.add(Dropout(0.2))
    
    #model2.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    #model2.add(BatchNormalization())
    #model2.add(Activation('relu'))
    #model2.add(Dropout(0.2))
    
    model1 = Sequential()
    model1.add(Merge(training_net1, mode ='concat'))
    model1.add(Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model1.add(Dropout(0.1))
    model1.add(BatchNormalization())
    model1.add(Activation('relu'))
    
    training_net.append(model1)
    training_net.append(model2)
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
    
    model.add(Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.1))
    if Dense5>0:
        model.add(Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
          
    return model

## Train and Evaluate the model

In [9]:
Data = load_data(165000,6000,5000)

26017
24222
37991
26023
24229
37993


In [9]:
def train_model(true_y = [],training = [], y = [], batch_size=4, epochs=12, verbose1 = 1, verbose2 = 1,
                              validation = [], val_y = [], testing = [], matrixsize11 = 6, nbfilter11 = 24, 
                              matrixsize12 = 7, nbfilter12 = 4, matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7,
                              nbfilter22 = 4, nbfilter1 = 96, kernel_size1 = 7, nbfilters2 = 96,kernel_size2 = 7,
                              Dense1 = 32, Dense2 = 192, Dense3 = 512, Dense4 = 64, Dense5 = 0):
    model = create_model2(matrixsize11, nbfilter11, matrixsize12, nbfilter12, matrixsize21, nbfilter21, 
                         matrixsize22, nbfilter22, nbfilter1, kernel_size1, nbfilters2,kernel_size2,
                         Dense1, Dense2, Dense3, Dense4, Dense5)
    earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose = verbose1)
    print('model training')
    model.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    # test   
    
    print('predicting')
    
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("batch_size : ", batch_size)
    print("epochs : ",epochs)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])
    
    return model


## Save The Model

In [10]:
def save_model(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model_double77-6.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model_double77-6.h5")
    json_file.close()
    print("Saved model to disk")

In [31]:
save_model(m_double_sens)

Saved model to disk


## Load The Model

In [12]:
def test_model(true_y = [],training = [], y = [], batch_size=4, epochs=12, verbose1 = 1, verbose2 = 1,
                              validation = [], val_y = [], testing = [], matrixsize11 = 6, nbfilter11 = 24, 
                              matrixsize12 = 7, nbfilter12 = 4, matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7,
                              nbfilter22 = 4, nbfilter1 = 96, kernel_size1 = 7, nbfilters2 = 96,kernel_size2 = 7,
                              Dense1 = 32, Dense2 = 192, Dense3 = 512, Dense4 = 64, Dense5 = 0):
    
    model = create_model2(matrixsize11, nbfilter11, matrixsize12, nbfilter12, matrixsize21, nbfilter21, 
                         matrixsize22, nbfilter22, nbfilter1, kernel_size1, nbfilters2,kernel_size2,
                         Dense1, Dense2, Dense3, Dense4, Dense5)
    return model

In [11]:
from keras.models import model_from_json
def load_model():
    # load json and create model
    json_file = open('model_double77-6.json', 'r')
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    json_file.close()
    # load weights into new model
    loaded_model.load_weights("model_double77-6.h5")
    print("Loaded model from disk")
    return loaded_model

In [13]:
m_test = test_model([],[],[],32,16,1,1,[],[],[],Dense1=128,Dense2=512,Dense3=512,Dense4=128,Dense5=64)

configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
(None, 3, 96)
(None, 3, 96)
(None, 3, 96)
(None, 3, 96)


/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/media/DATA/anaconda

In [14]:
m_test.load_weights("model_double77-6.h5")

## Run

In [29]:
m_double_sens = train_model(Data[5],Data[0],Data[1],32,16,1,1,Data[2],Data[3],Data[4],Dense1=128,Dense2=512,Dense3=512,Dense4=128,Dense5=64)

configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
(None, 3, 96)
(None, 3, 96)
(None, 3, 96)
(None, 3, 96)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:45: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:54: UserWarning: The `Merge` layer is deprecate

model training
Train on 165000 samples, validate on 6000 samples
Epoch 1/16
165000/165000 [==============================] - 144s 871us/step - loss: 0.5998 - val_loss: 0.5366
Epoch 2/16
165000/165000 [==============================] - 140s 846us/step - loss: 0.5411 - val_loss: 0.5049
Epoch 3/16
165000/165000 [==============================] - 140s 847us/step - loss: 0.5161 - val_loss: 0.4904
Epoch 4/16
165000/165000 [==============================] - 139s 843us/step - loss: 0.4989 - val_loss: 0.4914
Epoch 5/16
165000/165000 [==============================] - 140s 851us/step - loss: 0.4868 - val_loss: 0.4816
Epoch 6/16
165000/165000 [==============================] - 141s 852us/step - loss: 0.4782 - val_loss: 0.4702
Epoch 7/16
165000/165000 [==============================] - 141s 855us/step - loss: 0.4689 - val_loss: 0.4747
Epoch 8/16
165000/165000 [==============================] - 141s 855us/step - loss: 0.4602 - val_loss: 0.4636
Epoch 9/16
165000/165000 [=============================

165000/165000 [==============================] - 200s 1ms/step - loss: 0.3992 - val_loss: 0.4885
Epoch 11/16
165000/165000 [==============================] - 206s 1ms/step - loss: 0.3840 - val_loss: 0.4895
Epoch 00011: early stopping
predicting
[0.6334035  0.8119863  0.14555384 ... 0.45158726 0.13209444 0.66337335]
(array([1., 1., 0., ..., 0., 0., 1.], dtype=float32), array([1, 0.0, 0.0, ..., 0.0, 0.0, 1], dtype=object))
('batch_size : ', 16)
('epochs : ', 16)
##### ('acc : ', 0.7634)
('precision : ', 0.7690447400241838)
('sensitivity : ', 0.7577442414614773)
('specificity : ', 0.7691377921031426)
('MCC : ', 0.526883593223217)
m_double_sens = train_model(Data[5],Data[0],Data[1],16,16,1,1,Data[2],Data[3],Data[4],Dense1=128,Dense2=512,Dense3=512,Dense4=128,Dense5=64)

165000/165000 [==============================] - 129s 781us/step - loss: 0.4334 - val_loss: 0.4894
Epoch 8/16
165000/165000 [==============================] - 133s 806us/step - loss: 0.4205 - val_loss: 0.4754
Epoch 9/16
165000/165000 [==============================] - 136s 827us/step - loss: 0.4052 - val_loss: 0.4940
Epoch 10/16
165000/165000 [==============================] - 135s 817us/step - loss: 0.3903 - val_loss: 0.4835
Epoch 00010: early stopping
predicting
[0.08960374 0.35967925 0.08161914 ... 0.6650071  0.1162596  0.3793883 ]
(array([0., 0., 0., ..., 1., 0., 0.], dtype=float32), array([1, 0.0, 0.0, ..., 0.0, 0.0, 1], dtype=object))
('batch_size : ', 32)
('epochs : ', 16)
('acc : ', 0.77)
('precision : ', 0.8187325256290774)
('sensitivity : ', 0.6977760127084988)
('specificity : ', 0.8432715551974215)
('MCC : ', 0.5465405096248002)
m_double_sens = train_model(Data[5],Data[0],Data[1],32,16,1,1,Data[2],Data[3],Data[4],Dense1=128,Dense2=512,Dense3=512,Dense4=128,Dense5=64)

In [39]:
m_double_sens = train_model(Data[5],Data[0],Data[1],32,16,1,1,Data[2],Data[3],Data[4],Dense1=64,Dense2=256,Dense3=512,Dense4=256,Dense5=64)

configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
configure cnn network
(None, 10, 10, 24)
(None, 4, 4, 4)
(None, 7, 64)
(None, 7, 64)
(None, 7, 64)
(None, 7, 64)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


model training
Train on 165000 samples, validate on 6000 samples
Epoch 1/16
165000/165000 [==============================] - 128s 778us/step - loss: 0.5837 - val_loss: 0.5407
Epoch 2/16
165000/165000 [==============================] - 122s 741us/step - loss: 0.5291 - val_loss: 0.5263
Epoch 3/16
165000/165000 [==============================] - 127s 768us/step - loss: 0.4994 - val_loss: 0.5520
Epoch 4/16
165000/165000 [==============================] - 139s 843us/step - loss: 0.4746 - val_loss: 0.4933
Epoch 5/16
165000/165000 [==============================] - 130s 789us/step - loss: 0.4510 - val_loss: 0.4950
Epoch 6/16
165000/165000 [==============================] - 126s 764us/step - loss: 0.4290 - val_loss: 0.5004
Epoch 00006: early stopping
predicting
[0.345839   0.257545   0.08639022 ... 0.11804459 0.01557695 0.5377143 ]
(array([0., 0., 0., ..., 0., 0., 1.], dtype=float32), array([1, 0.0, 0.0, ..., 0.0, 0.0, 1], dtype=object))
('batch_size : ', 32)
('epochs : ', 16)
('acc : ', 0.755

In [16]:
help(keras.layers.convolutional.Deconv2D)

Help on class Conv2DTranspose in module keras.layers.convolutional:

class Conv2DTranspose(Conv2D)
 |  Transposed convolution layer (sometimes called Deconvolution).
 |  
 |  The need for transposed convolutions generally arises
 |  from the desire to use a transformation going in the opposite direction
 |  of a normal convolution, i.e., from something that has the shape of the
 |  output of some convolution to something that has the shape of its input
 |  while maintaining a connectivity pattern that is compatible with
 |  said convolution.
 |  
 |  When using this layer as the first layer in a model,
 |  provide the keyword argument `input_shape`
 |  (tuple of integers, does not include the sample axis),
 |  e.g. `input_shape=(128, 128, 3)` for 128x128 RGB pictures
 |  in `data_format="channels_last"`.
 |  
 |  # Arguments
 |      filters: Integer, the dimensionality of the output space
 |          (i.e. the number of output filters in the convolution).
 |      kernel_size: An intege